# Librerias

In [1]:
import numpy as np
import pandas as pd
import datetime
import warnings
import ast

warnings.filterwarnings("ignore")

# Configuración para que al imprimir las tablas se vean completas hasta 90 columnas
pd.options.display.max_columns = 90

# Paths

In [2]:
pathPrints = "../data/raw/prints.json"
pathTaps = "../data/raw/taps.json"
pathPayments = "../data/raw/pays.csv"

# Carga fuentes de datos

## Prints

In [3]:
df_prints = pd.read_json(pathPrints, lines=True)

In [4]:
print(f"Dimensiones del datase prints: {df_prints.shape}")
df_prints.head(5)

Dimensiones del datase prints: (508617, 3)


,day,event_data,user_id
0,2020-11-01,"{'position': 0, 'value_prop': 'cellphone_recha...",98702
1,2020-11-01,"{'position': 1, 'value_prop': 'prepaid'}",98702
2,2020-11-01,"{'position': 0, 'value_prop': 'prepaid'}",63252
3,2020-11-01,"{'position': 0, 'value_prop': 'cellphone_recha...",24728
4,2020-11-01,"{'position': 1, 'value_prop': 'link_cobro'}",24728


In [5]:
df_prints.info()

<class 'pandas.core.frame.DataFrame'>
Index: 508617 entries, 0 to 508616
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   day         508617 non-null  object
 1   event_data  508617 non-null  object
 2   user_id     508617 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 15.5+ MB


In [6]:
# Sacamos la informacion del Json de event_data a columnas
df_prints = df_prints.join(pd.json_normalize(df_prints["event_data"]))
df_prints["day"] = pd.to_datetime(df_prints["day"])

In [7]:
df_prints.head(5)

,day,event_data,user_id,position,value_prop
0,2020-11-01,"{'position': 0, 'value_prop': 'cellphone_recha...",98702,0,cellphone_recharge
1,2020-11-01,"{'position': 1, 'value_prop': 'prepaid'}",98702,1,prepaid
2,2020-11-01,"{'position': 0, 'value_prop': 'prepaid'}",63252,0,prepaid
3,2020-11-01,"{'position': 0, 'value_prop': 'cellphone_recha...",24728,0,cellphone_recharge
4,2020-11-01,"{'position': 1, 'value_prop': 'link_cobro'}",24728,1,link_cobro


## Taps

In [8]:
df_taps = pd.read_json(pathTaps, lines=True)

In [9]:
print(f"Dimensiones del dataset Taps: {df_taps.shape}")
df_taps.head(5)

Dimensiones del dataset Taps: (50859, 3)


,day,event_data,user_id
0,2020-11-01,"{'position': 0, 'value_prop': 'cellphone_recha...",98702
1,2020-11-01,"{'position': 2, 'value_prop': 'point'}",3708
2,2020-11-01,"{'position': 3, 'value_prop': 'send_money'}",3708
3,2020-11-01,"{'position': 0, 'value_prop': 'transport'}",93963
4,2020-11-01,"{'position': 1, 'value_prop': 'cellphone_recha...",93963


In [10]:
df_taps.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50859 entries, 0 to 50858
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   day         50859 non-null  object
 1   event_data  50859 non-null  object
 2   user_id     50859 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.6+ MB


In [11]:
# Sacamos la informacion del Json de event_data a columnas
df_taps = df_taps.join(pd.json_normalize(df_taps["event_data"]))
df_taps["day"] = pd.to_datetime(df_taps["day"])

In [12]:
df_taps.head(5)

,day,event_data,user_id,position,value_prop
0,2020-11-01,"{'position': 0, 'value_prop': 'cellphone_recha...",98702,0,cellphone_recharge
1,2020-11-01,"{'position': 2, 'value_prop': 'point'}",3708,2,point
2,2020-11-01,"{'position': 3, 'value_prop': 'send_money'}",3708,3,send_money
3,2020-11-01,"{'position': 0, 'value_prop': 'transport'}",93963,0,transport
4,2020-11-01,"{'position': 1, 'value_prop': 'cellphone_recha...",93963,1,cellphone_recharge


## Payments

In [13]:
df_pays = pd.read_csv(pathPayments)
df_pays = df_pays.rename(columns={"pay_date": "day"})
df_pays["day"] = pd.to_datetime(df_pays["day"])

In [14]:
print(f"Dimensiones del dataset Payments: {df_pays.shape}")
df_pays.head(5)

Dimensiones del dataset Payments: (756483, 4)


,day,total,user_id,value_prop
0,2020-11-01,7.04,35994,link_cobro
1,2020-11-01,37.36,79066,cellphone_recharge
2,2020-11-01,15.84,19321,cellphone_recharge
3,2020-11-01,26.26,19321,send_money
4,2020-11-01,35.35,38438,send_money


# Transformaciones Tablas

## Transformacion Prints

In [15]:
# Extraemos prints de la última semana
fecha_max = df_prints["day"].max()
fecha_inicio = fecha_max - pd.Timedelta(days=7)

df_prints_semana = df_prints[(df_prints["day"] > fecha_inicio) & (df_prints["day"] <= fecha_max)]\
.sort_values("day", ascending=False)\
.drop_duplicates(["user_id", "value_prop"])

In [16]:
print(f"Cantidad de prints unicos ultima semana: {df_prints_semana.shape[0]}")
df_prints_semana.head(5)

Cantidad de prints unicos ultima semana: 105819


,day,event_data,user_id,position,value_prop
508616,2020-11-30,"{'position': 0, 'value_prop': 'send_money'}",9462,0,send_money
498847,2020-11-30,"{'position': 1, 'value_prop': 'cellphone_recha...",58033,1,cellphone_recharge
498845,2020-11-30,"{'position': 0, 'value_prop': 'link_cobro'}",44874,0,link_cobro
498844,2020-11-30,"{'position': 0, 'value_prop': 'link_cobro'}",28806,0,link_cobro
498843,2020-11-30,"{'position': 0, 'value_prop': 'link_cobro'}",20913,0,link_cobro


In [17]:
# Prints de las 3 semanas anteriores a la semana actual

df_prints_sem3 = df_prints[df_prints["day"] < fecha_inicio]

prints_grouped = (
    df_prints_sem3.groupby(["user_id", "value_prop"])
             .agg(prints_count=("value_prop", "count"))
             .reset_index()
)

In [18]:
prints_grouped.head(5)

,user_id,value_prop,prints_count
0,1,credits_consumer,2
1,1,link_cobro,3
2,1,point,1
3,1,prepaid,1
4,1,send_money,1


## Transformación Taps

In [19]:
df_taps_semana = df_taps[["day", "user_id", "value_prop"]]
df_taps_semana["clic"] = 1

In [20]:
df_taps_semana.head(5)

,day,user_id,value_prop,clic
0,2020-11-01,98702,cellphone_recharge,1
1,2020-11-01,3708,point,1
2,2020-11-01,3708,send_money,1
3,2020-11-01,93963,transport,1
4,2020-11-01,93963,cellphone_recharge,1


In [21]:
# Clics realizados en las 3 semanas previas a la fecha de los prints
df_taps_sem3 = df_taps[df_taps["day"] < fecha_inicio]

taps_grouped = (
    df_taps_sem3.groupby(["user_id", "value_prop"])
             .agg(taps_count=("value_prop", "count"))
             .reset_index()
)

In [22]:
taps_grouped.head(5)

,user_id,value_prop,taps_count
0,1,link_cobro,1
1,3,point,1
2,4,link_cobro,1
3,7,send_money,1
4,12,link_cobro,1


## Transformación Payments

In [23]:
# Pagos realizados en las 3 semanas previas a la fecha de los prints

df_pays_sem3 = df_pays[df_pays["day"] < fecha_inicio]

In [24]:
pays_grouped = (
    df_pays_sem3.groupby(["user_id", "value_prop"])
             .agg(pays_count=("value_prop", "count"), pays_total=("total", "sum"))
             .reset_index()
)

In [25]:
pays_grouped.head(5)

,user_id,value_prop,pays_count,pays_total
0,1,credits_consumer,1,37.92
1,1,transport,1,100.89
2,2,link_cobro,2,94.10
3,2,point,2,173.55
4,2,send_money,1,84.78


In [26]:
df_pays_semana = df_taps[["day", "user_id", "value_prop"]]
df_pays_semana["label"] = 1

## Estructura final para modelos

In [27]:
df_final = df_prints_semana.merge(df_taps_semana, on = ["day", "user_id", "value_prop"], how="left")\
.merge(prints_grouped, on=["user_id", "value_prop"], how="left")\
.merge(taps_grouped, on=["user_id", "value_prop"], how="left")\
.merge(pays_grouped, on=["user_id", "value_prop"], how="left")\
.drop(["event_data"], axis=1)

In [28]:
for column in df_final.columns[4:]:

    if column != "pays_total":
        df_final[column] = df_final[column].fillna(0).astype(int)
    else:
        df_final[column] = df_final[column].fillna(0)



In [29]:
print(f"Dimensiones del dataset final: {df_final.shape}")
df_final.head(5)

Dimensiones del dataset final: (105819, 9)


,day,user_id,position,value_prop,clic,prints_count,taps_count,pays_count,pays_total
0,2020-11-30,9462,0,send_money,0,1,0,0,0.00
1,2020-11-30,58033,1,cellphone_recharge,0,1,0,0,0.00
2,2020-11-30,44874,0,link_cobro,0,2,0,0,0.00
3,2020-11-30,28806,0,link_cobro,0,1,0,2,111.52
4,2020-11-30,20913,0,link_cobro,0,1,0,0,0.00


# Base 1: Modelo Clasificacion

In [30]:
df_mod1 = df_final.merge(df_pays_semana, on = ["day", "user_id", "value_prop"], how="left")
df_mod1["label"] = df_mod1["label"].fillna(0).astype(int)

In [31]:
df_mod1.head(5)

,day,user_id,position,value_prop,clic,prints_count,taps_count,pays_count,pays_total,label
0,2020-11-30,9462,0,send_money,0,1,0,0,0.00,0
1,2020-11-30,58033,1,cellphone_recharge,0,1,0,0,0.00,0
2,2020-11-30,44874,0,link_cobro,0,2,0,0,0.00,0
3,2020-11-30,28806,0,link_cobro,0,1,0,2,111.52,0
4,2020-11-30,20913,0,link_cobro,0,1,0,0,0.00,0


## Exportar base modelo de clasificacion

In [32]:
df_mod1.to_csv("../data/processed/clasification_model_df.csv", index=False)

# Base 2: Modelo de recomendacion Apriori (Canasta de compra)

In [33]:
# Extraer usuarios unicos

df_users_prints = df_prints[["user_id"]].copy()
df_users_prints = df_users_prints.drop_duplicates()

df_users_taps = df_taps[["user_id"]].copy()
df_users_taps = df_users_taps.drop_duplicates()

df_users_pays = df_pays[["user_id"]].copy()
df_users_pays = df_users_pays.drop_duplicates()

In [34]:
df_users = pd.concat([df_users_prints, df_users_taps, df_users_pays], ignore_index=True)\
.drop_duplicates()

In [35]:
basket_df = df_pays.assign(flag=1)\
.drop_duplicates(['user_id', 'value_prop'])\
.pivot(index='user_id', columns='value_prop', values='flag')\
.fillna(0)\
.astype(int)\
.reset_index()\
.rename_axis(None, axis=1)



In [36]:
basket_df.head(5)

,user_id,cellphone_recharge,credits_consumer,link_cobro,point,prepaid,send_money,transport
0,1,1,1,1,0,0,0,1
1,2,0,1,1,1,1,1,1
2,3,1,0,1,1,1,0,1
3,4,1,1,0,1,0,1,0
4,5,0,1,1,1,1,1,0


In [37]:
df_mod2 = df_users.merge(basket_df, on=["user_id"], how="left")\
.fillna(0)\
.astype(int)

In [38]:
print(f"Cantidad de clientes: {df_mod2.shape[0]}")
df_mod2.head(5)

Cantidad de clientes: 99943


,user_id,cellphone_recharge,credits_consumer,link_cobro,point,prepaid,send_money,transport
0,98702,0,0,1,1,1,0,0
1,63252,1,0,0,0,0,1,0
2,24728,0,0,1,0,0,1,1
3,25517,1,1,0,1,0,1,0
4,57587,1,0,1,0,0,1,1


## Exportar base modelo de recomendacion

In [39]:
df_mod2.to_csv("../data/processed/apriori_model_df.csv", index=False)

# Base 3: Modelo de recomendacion filtrado colaborativo (SVD)

In [40]:
df_mod3 = df_final.copy()
      
df_mod3["rating"] = (
    0.5 * df_mod3["pays_total"].apply(lambda x: np.log1p(x)) +
    0.25 * df_mod3["pays_count"] +
    0.15 * df_mod3["taps_count"] +
    0.10 * df_mod3["prints_count"] 
)

In [41]:
print(f"Dimensiones del dataset modelo 3: {df_mod3.shape}")
df_mod3.head(5)

Dimensiones del dataset modelo 3: (105819, 10)


,day,user_id,position,value_prop,clic,prints_count,taps_count,pays_count,pays_total,rating
0,2020-11-30,9462,0,send_money,0,1,0,0,0.00,0.100000
1,2020-11-30,58033,1,cellphone_recharge,0,1,0,0,0.00,0.100000
2,2020-11-30,44874,0,link_cobro,0,2,0,0,0.00,0.200000
3,2020-11-30,28806,0,link_cobro,0,1,0,2,111.52,2.961565
4,2020-11-30,20913,0,link_cobro,0,1,0,0,0.00,0.100000


In [42]:
df_mod3 = df_mod3.pivot(index="user_id", columns="value_prop", values="rating").fillna(0)

In [43]:
print(f"Dimensiones del dataset: {df_mod3.shape}")
df_mod3.head(5)

Dimensiones del dataset: (37410, 7)


value_prop,cellphone_recharge,credits_consumer,link_cobro,point,prepaid,send_money,transport
user_id,,,,,,,
1,0.0,0.0,0.450000,0.100000,0.000000,0.000000,2.761947
2,0.1,0.0,0.000000,3.081106,0.000000,2.475893,0.000000
3,0.0,0.0,2.781171,0.000000,3.316367,0.000000,0.000000
4,0.0,0.0,0.350000,0.000000,0.100000,0.000000,0.000000
6,0.1,0.0,0.000000,0.000000,1.046654,2.413785,0.000000


## Exportar base modelo de recomendacion 2

In [44]:
df_mod3.to_csv("../data/processed/SVD_model_df.csv")